In [10]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()


Node 848cf03d-8bca-4685-9cca-f00b771ebd9e

In [1]:
import loc_avoid

titi = loc_avoid.Robot2()



In [5]:
#Correct code
import time


local_state = 1
DEFAULT = 0
FRONT = 1
FRONT_LEFT = 2
FRONT_RIGHT = 3
RIGHT_OBST = 4
LEFT_OBST = 5
CRITIC = 6
OBST_DETECTED = 7

rotationThr = 2000
obstThrH = 3000
speed = 0

def update_prox_values(node):
    aw(node.wait_for_variables({"prox.horizontal"}))
    prox = list(node.v.prox.horizontal).copy()
    return prox

def set_speed(node, left,right):
    v = {"motor.left.target": [left],
        "motor.right.target": [right],
    }
    aw(node.set_variables(v))

def update_local_state(prox):

    global local_state, obstThrH
    
    obst_detected = [0,0,0,0,0]
    sum_obst = 0

    for i in range(5):  # Parcourt chaque capteur de proximité
        if prox[i] > obstThrH:  # Si l'obstacle est détecté à l'un des capteurs
            local_state = OBST_DETECTED
            obst_detected[i] = 1
    #if rien tu sors
    for i in range(5):
        sum_obst = sum_obst + obst_detected[i]
    if sum_obst == 0:
        local_state = DEFAULT
        return
    
    #if obst detected 
    if obst_detected[2] == 1:
        local_state = FRONT
    elif obst_detected[0] == 1 or obst_detected[1] == 1: 
        local_state = LEFT_OBST
    elif obst_detected[3] == 1 or obst_detected[4] == 1:
        local_state = RIGHT_OBST  

def front_obst(node, prox):     #faire update state qui set deja le moteur de base pour chaque etat, et selon etat appelé une fonction

    global local_state
    if local_state not in {1, 2, 3}:
        return 
    
    close_left = (prox[0] + prox[1]) // 2
    close_right = (prox[3] + prox[4]) // 2

    if local_state == FRONT:
        if close_left > close_right:
            local_state = FRONT_LEFT
            
        elif close_right > close_left:
            local_state = FRONT_RIGHT
            
    if local_state == FRONT_LEFT:
        print(local_state)
        while close_left > rotationThr:
            set_speed(node, speed, -speed)
            prox = update_prox_values(node)
            print("close_left =", close_left)
            close_left = (prox[0] + prox[1]) // 2
        print("close_left =", close_left)
        local_state = DEFAULT

    elif local_state == FRONT_RIGHT:
        while close_right > rotationThr:
            set_speed(node, -speed, speed)
            prox = update_prox_values(node)
            close_right = (prox[3] + prox[4]) // 2
        local_state = DEFAULT
   
def side_obst(node, prox):

    global local_state
    if local_state not in {4,5}:
        return
    
    if local_state == LEFT_OBST:
        while prox[0] > rotationThr:
            set_speed(node, speed, -speed)
            prox = update_prox_values(node)
        local_state = DEFAULT

    if local_state == RIGHT_OBST:
        while prox[4] > rotationThr:
            set_speed(node, -speed, speed)
            prox = update_prox_values(node)
        local_state = DEFAULT

    

while True:
    obst = update_prox_values(node)
    
    set_speed(node, speed, speed)
    update_local_state(obst)
    #time.sleep(0.5)
    front_obst(node, obst)
    side_obst(node, obst)
   
    aw(node.wait_for_variables({"prox.horizontal"}))
    print("state :", local_state, obst)

   

state : 0 [0, 1463, 1908, 0, 0, 0, 0]
state : 0 [0, 1470, 1915, 0, 0, 0, 0]
state : 0 [0, 1459, 1904, 0, 0, 0, 0]
state : 0 [0, 1458, 1915, 0, 0, 0, 0]
state : 0 [0, 1464, 1918, 0, 0, 0, 0]
state : 0 [0, 1465, 1916, 0, 0, 0, 0]
state : 0 [0, 1461, 1919, 0, 0, 0, 0]
state : 0 [0, 1462, 1918, 0, 0, 0, 0]
state : 0 [0, 1462, 1918, 0, 0, 0, 0]
state : 0 [0, 1459, 1918, 0, 0, 0, 0]
state : 0 [0, 1465, 1915, 0, 0, 0, 0]
state : 0 [0, 1459, 1917, 0, 0, 0, 0]
state : 0 [0, 1456, 1919, 0, 0, 0, 0]
state : 0 [0, 1454, 1915, 0, 0, 0, 0]
state : 0 [0, 1461, 1919, 0, 0, 0, 0]
state : 0 [0, 1466, 1918, 0, 0, 0, 0]
state : 0 [0, 1464, 1912, 0, 0, 0, 0]
state : 0 [0, 1463, 1916, 0, 0, 0, 0]
state : 0 [0, 1463, 1918, 0, 0, 0, 0]
state : 0 [0, 1467, 1917, 0, 0, 1793, 2371]
state : 0 [0, 1531, 1902, 0, 0, 3574, 0]
state : 0 [0, 2324, 1879, 0, 0, 4437, 0]
state : 0 [0, 2324, 1879, 0, 0, 4437, 0]
2
close_left = 3873
close_left = 3978
close_left = 3771
close_left = 3439
close_left = 3753
close_left = 4007
c

KeyboardInterrupt: 

In [16]:
#Correct code with Robot class
import loc_avoid

# local_state with the differents states for local avoidance 
local_state = 1
DEFAULT = 0
FRONT = 1
FRONT_LEFT = 2
FRONT_RIGHT = 3
RIGHT_OBST = 4
LEFT_OBST = 5
CRITIC = 6
OBST_DETECTED = 7

# Different cste for threshold, speed, rotation threshold
rotationThr = 2000
obstThrH = 3000
speed = 150

# ENLEVER global instance : titi the little thymio instance 
robot_instance = loc_avoid.Robot2()

# Function update proximity values for Thymio
def update_prox_values(node):
    aw(node.wait_for_variables({"prox.horizontal"}))
    prox = list(node.v.prox.horizontal).copy()
    return prox

# Function to set speed of motors
def set_speed(node, left,right):
    v = {"motor.left.target": [left],
        "motor.right.target": [right],
    }
    aw(node.set_variables(v))

# Function to update current local state
def update_local_state():

    global local_state, obstThrH, robot_instance
    
    obst_detected = [0,0,0,0,0]
    sum_obst = 0

    for i in range(5):  # Parcourt chaque capteur de proximité
        #if prox[i] > obstThrH:  # Si l'obstacle est détecté à l'un des capteurs
        if robot_instance.front_prox[i] > obstThrH:  # Si l'obstacle est détecté à l'un des capteurs
            local_state = OBST_DETECTED
            obst_detected[i] = 1
    #if rien tu sors
    for i in range(5):
        sum_obst = sum_obst + obst_detected[i]
    if sum_obst == 0:
        local_state = DEFAULT
        return
    
    #if obst detected 
    if obst_detected[2] == 1:
        local_state = FRONT
    elif obst_detected[0] == 1 or obst_detected[1] == 1: 
        local_state = LEFT_OBST
    elif obst_detected[3] == 1 or obst_detected[4] == 1:
        local_state = RIGHT_OBST  

# Front obstacle reactions function
def front_obst(node):     

    global local_state, robot_instance
    if local_state not in {1, 2, 3}:
        return 
    
    close_left = (robot_instance.front_prox[0] + robot_instance.front_prox[1]) // 2
    close_right = (robot_instance.front_prox[3] + robot_instance.front_prox[4]) // 2

    if local_state == FRONT:
        if close_left > close_right:
            local_state = FRONT_LEFT
            
        elif close_right > close_left:
            local_state = FRONT_RIGHT
            
    if local_state == FRONT_LEFT:
        print(local_state)
        while close_left > rotationThr:
            set_speed(node, speed, -speed)
            robot_instance.update_front_prox(update_prox_values(node))
            close_left = (robot_instance.front_prox[0] + robot_instance.front_prox[1]) // 2
        local_state = DEFAULT

    elif local_state == FRONT_RIGHT:
        while close_right > rotationThr:
            set_speed(node, -speed, speed)
            robot_instance.update_front_prox(update_prox_values(node))
            close_right = (robot_instance.front_prox[3] + robot_instance.front_prox[4]) // 2
        local_state = DEFAULT

# Side obstacle reactions function 
def side_obst(node):

    global local_state, robot_instance
    if local_state not in {4,5}:
        return
    
    if local_state == LEFT_OBST:
        while robot_instance.front_prox[0] > rotationThr:
            set_speed(node, speed, -speed)
            robot_instance.update_front_prox(update_prox_values(node))
        local_state = DEFAULT

    if local_state == RIGHT_OBST:
        while robot_instance.front_prox[4] > rotationThr:
            set_speed(node, -speed, speed)
            robot_instance.update_front_prox(update_prox_values(node))
        local_state = DEFAULT

    
def local_avoidance(node, speed):

    # Updating new values for proximity sensors
    robot_instance.update_front_prox(update_prox_values(node))  

    # Setting default speed 
    set_speed(node, speed, speed)

    # Updating local state to do the corresponding avoidance
    update_local_state()  

    # Local avoidance functions for front and side obstacle called
    front_obst(node)    
    side_obst(node)

    # Waiting for new Thymio's proximity data 
    aw(node.wait_for_variables({"prox.horizontal"}))    # Waiting for new Thymio's proximity data 

# Local main 
while True:
    # local avoid function
    local_avoidance(node, speed)


   

KeyboardInterrupt: 